In [ ]:
import os
import tabula
import numpy as np

import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import plotly.express as px
pd.options.plotting.backend = "plotly"

### Load B3 Setor

In [ ]:
setorial_path = os.path.join("data", "b3_setorial.csv")
df_setorial = pd.read_csv(
    setorial_path, sep=";", encoding="latin"
)

In [ ]:
df_setorial.sample(n=5)

### Ações

In [ ]:
df_acoes = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Acoes')

In [ ]:
df_acoes.drop(columns=['Conta', 'Código ISIN / Distribuição', 'Escriturador', 
    'Quantidade Disponível', 'Quantidade Indisponível', 
    'Motivo', 'Preço de Fechamento'], inplace=True)

df_acoes.rename(
    columns={
        'Produto' : 'des_produto',
        'Instituição' : 'des_conta',
        'Código de Negociação' : 'des_movimentacao',
        'Código de Negociação' : 'cod_acao',
        'Tipo' : 'tp_acao',
        'Quantidade' : 'quantidade',
        'Valor Atualizado' : 'vlr_total'
    }, 
    inplace=True
)

df_acoes = df_acoes[(~df_acoes['des_produto'].isna()) & (~df_acoes['des_conta'].isna())]

In [ ]:
df_acoes['des_produto'] = df_acoes['des_produto'].str.lstrip().str.rstrip()
df_acoes['des_produto'] = df_acoes['des_produto'].str[7:]
df_acoes['des_produto'] = df_acoes['des_produto'].str.replace('- TRANSMISSORA', 'TRANSMISSORA')
df_acoes['vlr_total'] = df_acoes['vlr_total'].astype(np.float32)

In [ ]:
# Merge Setor
df_acoes['tmp_cod_acap'] = df_acoes['cod_acao'].str[0:4]
df_acoes = pd.merge(df_acoes, df_setorial, left_on='tmp_cod_acap', right_on='codigo')

In [ ]:
# Percentual da acao na carteira
df_acoes['tmp_vlr_total'] = df_acoes['vlr_total'].sum()
df_acoes['pct_vlr_total'] = (df_acoes['vlr_total'] / df_acoes['tmp_vlr_total']) * 100

In [ ]:
df_group_setor = df_acoes[['setor','vlr_total']].groupby(['setor']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_setor'})
def_group_subsetor = df_acoes[['subsetor','vlr_total']].groupby(['subsetor']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_subsetor'})
df_group_segmento = df_acoes[['segmento','vlr_total']].groupby(['segmento']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_segmento'})

df_acoes = pd.merge(df_acoes, df_group_setor, on='setor')
df_acoes = pd.merge(df_acoes, def_group_subsetor, on='subsetor')
df_acoes = pd.merge(df_acoes, df_group_segmento, on='segmento')

In [ ]:
# Percentual do setor/subsetor/segmento da carteira
df_acoes['pct_vlr_total_setor'] = (df_acoes['vlr_total_setor'] / df_acoes['tmp_vlr_total']) * 100
df_acoes['pct_vlr_total_subsetor'] = (df_acoes['vlr_total_subsetor'] / df_acoes['tmp_vlr_total']) * 100
df_acoes['pct_vlr_total_segmento'] = (df_acoes['vlr_total_segmento'] / df_acoes['tmp_vlr_total']) * 100

In [ ]:
columns = ['des_conta', 'cod_acao', 'des_produto', 
        'tp_acao', 'setor', 'subsetor', 'segmento',
        'quantidade', 'vlr_total', 'pct_vlr_total',
        'vlr_total_setor', 'pct_vlr_total_setor',
        'vlr_total_subsetor', 'pct_vlr_total_subsetor',
        'vlr_total_segmento', 'pct_vlr_total_segmento']
        
df_acoes = df_acoes[columns]

In [ ]:
df_acoes

In [ ]:
px.treemap(
    df_acoes, 
    path=["setor", "subsetor", "segmento", "cod_acao"], 
    values="pct_vlr_total",
    width=1000, height=800
    )

### FII

In [ ]:
df_fundos = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Fundo de Investimento')

In [ ]:
df_fundos

In [ ]:
df_fundos.drop(
    columns=[
        'Conta', 'Código ISIN / Distribuição', 'Administrador', 
        'Quantidade Disponível', 'Quantidade Indisponível', 
        'Motivo', 'Preço de Fechamento', 'Tipo'
    ], inplace=True
)

df_fundos.rename(
    columns={
        'Produto' : 'des_produto',
        'Instituição' : 'des_conta',
        'Código de Negociação' : 'des_movimentacao',
        'Código de Negociação' : 'cod_acao',
        'Quantidade' : 'quantidade',
        'Valor Atualizado' : 'vlr_total'
    }, 
    inplace=True
)

df_fundos = df_fundos[(~df_fundos['des_produto'].isna()) & (~df_fundos['des_conta'].isna())]

In [ ]:
# create a dataframe
marks = pd.DataFrame({'Name': ['Akhil', 'Sai', 'Rohit', 'Prasanth', 'Divya'],
                      'Percentage': ['-', 65, 90, 79, 89],
                      'Grade': ['-', 'C', 'O', 'B', 'A']})
 
# Assign Absent if percentage is not specified
marks[marks.Percentage == '-'].Grade = 'Ab'

In [ ]:
df_fundos['vlr_total'] = df_fundos['vlr_total'].astype(np.float32)

# Percentual da acao na carteira
df_fundos['tmp_vlr_total'] = df_fundos.loc[:,'vlr_total'].sum()
df_fundos['pct_vlr_total'] = (df_fundos.loc[:,'vlr_total'] / df_fundos.loc[:,'tmp_vlr_total']) * 100

In [ ]:
df_fundos

In [ ]:
tp_fii = {
    'BTLG11' : 'Tijolo',
    'KNRI11' : 'Tijolo',
    'XPML11' : 'Tijolo',
    'KNCA11' : 'Papel',
    'OUJP11' : 'Papel'
}

df_fundos['tp_fii'] = df_fundos['cod_acao'].map(tp_fii)

In [ ]:
segmento = {
    'BTLG11' : 'Logística',
    'KNRI11' : 'Híbrido',
    'XPML11' : 'Shoppings',
    'KNCA11' : 'Títulos e Valores Mobiliários',
    'OUJP11' : 'Títulos e Valores Mobiliários'
}

df_fundos['segmento'] = df_fundos['cod_acao'].map(segmento)

In [ ]:
df_fundos

In [ ]:
px.treemap(
    df_fundos, 
    path=["tp_fii", "segmento", "cod_acao"], 
    values="pct_vlr_total",
    width=1000, height=800
    )

### Tesouro Direto

In [ ]:
df_tmp_tesouro_direto = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Tesouro Direto')

In [ ]:
df_tesouro_direto = df_tmp_tesouro_direto.copy()

df_tesouro_direto.drop(columns=['Instituição', 'Código ISIN', 'Quantidade Disponível', 
    'Quantidade Indisponível', 'Motivo', 'Valor Atualizado'], inplace=True)

df_tesouro_direto.rename(
    columns={
        'Produto' : 'des_produto',
        'Indexador' : 'indexador',
        'Vencimento' : 'vencimento',
        'Quantidade' : 'quantidade',
        'Valor Aplicado' : 'vlr_aplicado',
        'Valor bruto' : 'vlr_bruto',
        'Valor líquido' : 'vlr_liquido'
    }, 
    inplace=True
)

df_tesouro_direto = df_tesouro_direto[(~df_tesouro_direto['des_produto'].isna()) & (~df_tesouro_direto['indexador'].isna())]

In [ ]:
# Percentual da acao na carteira
df_tesouro_direto['tmp_vlr_liquido_total'] = df_tesouro_direto['vlr_liquido'].sum()
df_tesouro_direto['pct_vlr_liquido_total'] = (df_tesouro_direto['vlr_liquido'] / df_tesouro_direto['tmp_vlr_liquido_total']) * 100

In [ ]:
df_tesouro_direto

In [ ]:
px.treemap(
    df_tesouro_direto, 
    path=["indexador", "des_produto"], 
    values="pct_vlr_liquido_total",
    width=1000, height=800
    )

## BANCO INTER

### Fundos de Investimentos

In [ ]:
tables_fundos =  tabula.read_pdf('data/extrato-posicao-fundos.pdf')

In [ ]:
novas_colunas =  ["des_produto", "dt_cotacao", "qt_cota",
    "vlr_cota", "vlr_aplicado", "vlr_bruto", "vlr_ir_previsto",
    "vlr_iof_previsto", "vlr_liquido"]

des_produto = tables_fundos[2].columns[0]
novos_valores = tables_fundos[2].iloc[1].values.tolist()
novos_valores.insert(0, des_produto)

In [ ]:
df_fundos = pd.DataFrame(columns=novas_colunas, data=[novos_valores])

In [ ]:
novas_colunas =  ["des_produto", "dt_cotacao", "qt_cota",
    "vlr_cota", "vlr_aplicado", "vlr_bruto", "vlr_ir_previsto",
    "vlr_iof_previsto", "vlr_liquido"]

des_produto = tables_fundos[3].columns[0]    
novos_valores = tables_fundos[3].iloc[1].values.tolist()
novos_valores.insert(0, des_produto)

In [ ]:
df_fundos = df_fundos.append(pd.DataFrame(columns=novas_colunas, data=[novos_valores]))

#### Transformações

In [ ]:
df_fundos['des_produto'] = df_fundos['des_produto'].str.lstrip().str.rstrip()
df_fundos['dt_cotacao'] = pd.to_datetime(df_fundos['dt_cotacao'], format='%d/%m/%Y')
df_fundos['qt_cota'] = df_fundos['qt_cota'].astype(np.float32)

df_fundos['vlr_cota'] = df_fundos['vlr_cota'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_cota'] = df_fundos['vlr_cota'].astype(np.float32)

df_fundos['vlr_aplicado'] = df_fundos['vlr_aplicado'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_aplicado'] = df_fundos['vlr_aplicado'].astype(np.float32)

df_fundos['vlr_bruto'] = df_fundos['vlr_bruto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_bruto'] = df_fundos['vlr_bruto'].astype(np.float32)

df_fundos['vlr_ir_previsto'] = df_fundos['vlr_ir_previsto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_ir_previsto'] = df_fundos['vlr_ir_previsto'].astype(np.float32)

df_fundos['vlr_iof_previsto'] = df_fundos['vlr_iof_previsto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_iof_previsto'] = df_fundos['vlr_iof_previsto'].astype(np.float32)

df_fundos['vlr_liquido'] = df_fundos['vlr_liquido'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_fundos['vlr_liquido'] = df_fundos['vlr_liquido'].astype(np.float32)

In [ ]:
df_fundos

### Renda Fixa

In [ ]:
tables =  tabula.read_pdf('data/extrato-posicao-renda-fixa.pdf',lattice=True)

In [ ]:
df_tmp_renda_fixa = tables[3][~tables[3]['Nota'].isna()]

In [ ]:
des_produto = tables[2].columns[0]

df_tmp_renda_fixa.drop(columns=["Nota"], inplace=True)
df_tmp_renda_fixa['des_produto'] = des_produto

df_tmp_renda_fixa.rename(
    columns={
        'Data Início' : 'dt_inicio',
        'Data\rVencimento' : 'dt_vencimento',
        'Valor\rAplicação' : 'vlr_aplicado',
        'Tipo\rAplicação' : 'tp_aplicacao',
        'Taxa\rAplicação' : 'taxa_aplicao',
        'Valor\rRendimento' : 'vlr_rendimento',
        'Valor\rRetirada' : 'vlr_retirada', 
        'Valor\rDesconto' : 'vlr_desconto',
        'Valor\rBruto' : 'vlr_bruto',
        'Valor Previsão\rDesconto' : 'vlr_previsao_desconto',
        'Valor\rLíquido' : 'vlr_liquido',
        'IR/IOF' : 'vlr_ir_iof'    
    }, 
    inplace=True
)

df_tmp_renda_fixa = df_tmp_renda_fixa[[
    "des_produto", "dt_inicio", 'dt_vencimento', 'vlr_aplicado',
    'tp_aplicacao', 'taxa_aplicao', 'vlr_rendimento',
    'vlr_retirada', 'vlr_desconto', 'vlr_bruto', 
    'vlr_previsao_desconto', 'vlr_liquido', 'vlr_ir_iof'
]].copy()

In [ ]:
df_tmp_renda_fixa

In [ ]:
df_tmp_renda_fixa['des_produto'] = df_tmp_renda_fixa['des_produto'].str.lstrip().str.rstrip()
df_tmp_renda_fixa['tp_aplicacao'] = df_tmp_renda_fixa['tp_aplicacao'].str.lstrip().str.rstrip()
df_tmp_renda_fixa['taxa_aplicao'] = df_tmp_renda_fixa['taxa_aplicao'].str.lstrip().str.rstrip()

df_tmp_renda_fixa['vlr_rendimento'] = df_tmp_renda_fixa['vlr_rendimento'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_rendimento'] = df_tmp_renda_fixa['vlr_rendimento'].astype(np.float32)

df_tmp_renda_fixa['vlr_retirada'] = df_tmp_renda_fixa['vlr_retirada'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_retirada'] = df_tmp_renda_fixa['vlr_retirada'].astype(np.float32)

df_tmp_renda_fixa['vlr_desconto'] = df_tmp_renda_fixa['vlr_desconto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_desconto'] = df_tmp_renda_fixa['vlr_desconto'].astype(np.float32)

df_tmp_renda_fixa['vlr_bruto'] = df_tmp_renda_fixa['vlr_bruto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_bruto'] = df_tmp_renda_fixa['vlr_bruto'].astype(np.float32)

df_tmp_renda_fixa['vlr_previsao_desconto'] = df_tmp_renda_fixa['vlr_previsao_desconto'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_previsao_desconto'] = df_tmp_renda_fixa['vlr_previsao_desconto'].astype(np.float32)

df_tmp_renda_fixa['vlr_liquido'] = df_tmp_renda_fixa['vlr_liquido'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_liquido'] = df_tmp_renda_fixa['vlr_liquido'].astype(np.float32)

df_tmp_renda_fixa['vlr_ir_iof'] = df_tmp_renda_fixa['vlr_ir_iof'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_tmp_renda_fixa['vlr_ir_iof'] = df_tmp_renda_fixa['vlr_ir_iof'].astype(np.float32)

In [ ]:
df_tmp_renda_fixa

## XP

In [ ]:
df_xp = pd.read_excel('data/PosicaoDetalhada.xlsx')

In [ ]:
df_xp

In [ ]:
df_xp['indexador'] = df_xp.columns[0]
df_xp.rename(columns={
    df_xp.columns[0] : 'des_produto',
    'Posição' : 'vlr_bruto',
    '% Alocação' : 'pct_alocacao',
    'Rentabilidade' : 'pct_rentabilidade',
    'Valor aplicado' : 'vlr_aplicado',
    'Valor líquido' : 'vlr_liquido',
    'Data da cota' : 'dt_cotacao'
    }, 
    inplace=True
)

In [ ]:
df_tesouro_direto

In [ ]:
df_fundos

In [ ]:
df_xp

In [ ]:
df_xp['des_produto'] = df_xp['des_produto'].str.lstrip().str.rstrip()
df_xp['indexador'] = df_xp['indexador'].str.lstrip().str.rstrip()

df_xp['dt_cotacao'] = pd.to_datetime(df_xp['dt_cotacao'], format='%d/%m/%Y')

df_xp['pct_alocacao'] = df_xp['pct_alocacao'].str.replace('%', '').str.replace('.','').str.replace(',','.')
df_xp['pct_alocacao'] = df_xp['pct_alocacao'].astype(np.float32)

df_xp['pct_rentabilidade'] = df_xp['pct_rentabilidade'].str.replace('%', '').str.replace('.','').str.replace(',','.')
df_xp['pct_rentabilidade'] = df_xp['pct_rentabilidade'].astype(np.float32)

df_xp['vlr_aplicado'] = df_xp['vlr_aplicado'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_xp['vlr_aplicado'] = df_xp['vlr_aplicado'].astype(np.float32)

df_xp['vlr_liquido'] = df_xp['vlr_liquido'].str.replace('R\$', '').str.replace('.','').str.replace(',','.')
df_xp['vlr_liquido'] = df_xp['vlr_liquido'].astype(np.float32)

In [ ]:
df_xp

## WARREN

### Renda Fixa

In [ ]:
df_warren_renda_fixa = pd.read_excel('data/warren-2022-01.xlsx', sheet_name='Renda Fixa')

In [ ]:
df_warren_renda_fixa

### Renda Variável

In [ ]:
df_warren_renda_variavel = pd.read_excel('data/warren-2022-01.xlsx', sheet_name='Renda Variavel')

In [ ]:
df_warren_renda_variavel

## Epiricus

In [ ]:
df_empiricus = pd.read_excel('data/empiricus_2022-01.xlsx')

In [ ]:
df_empiricus